# PyKale Tutorial: Domain Adaptation for Autism Detection with Multi-site Brain Imaging Data
| [Open in Colab](https://colab.research.google.com/github/pykale/pykale/blob/main/examples/multisite_neuroimg_adapt/tutorial.ipynb) (click `Runtime` → `Run all (Ctrl+F9)` |  [Launch Binder](https://mybinder.org/v2/gh/pykale/pykale/HEAD?filepath=examples%2Fmultisite_neuroimg_adapt%2Ftutorial.ipynb) (click `Run` → `Run All Cells`) |

## Overview

- Pre-processing:
    - [Data loading](#Data-Preparation)
    - [Construct brain networks](#Extracting-Brain-Networks-Features)
- Machine learning pipeline:
    - [Baseline: Ridge classifier](#Baseline-Model)
    - [Domain adaptation](#Domain-Adaptation)

**Reference:**

[1] Craddock C., Benhajali Y., Chu C., Chouinard F., Evans A., Jakab A., Khundrakpam BS., Lewis JD., Li Q., Milham M., Yan C. and Bellec P. (2013). [The Neuro Bureau Preprocessing Initiative: Open Sharing of Preprocessed Neuroimaging Data and Derivatives](https://doi.org/10.3389/conf.fninf.2013.09.00041). Frontiers in Neuroinformatics, 7.

[2] Abraham A., Pedregosa F., Eickenberg M., Gervais P., Mueller A., Kossaifi J., Gramfort A., Thirion B. and Varoquaux G. (2014). [Machine Learning for Neuroimaging with scikit-learn](https://doi.org/10.3389/fninf.2014.00014). Frontiers in Neuroinformatics, 8.

[3] Zhou S., Li W., Cox C. and Lu H. (2020). [Side Information Dependence as a Regularizer for Analyzing Human Brain Conditions across Cognitive Experiments](https://doi.org/10.1609/aaai.v34i04.6179). Proceedings of the AAAI Conference on Artificial Intelligence, 34(04), 6957-6964.

[4] Zhou S. (2022). [Interpretable Domain-Aware Learning for Neuroimage Classification](https://etheses.whiterose.ac.uk/31044/1/PhD_thesis_ShuoZhou_170272834.pdf) (Doctoral Dissertation, University of Sheffield).

## Setup

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !pip uninstall --yes imgaug && pip uninstall --yes albumentations && pip install git+https://github.com/aleju/imgaug.git
    !git clone https://github.com/pykale/pykale.git
    %cd pykale
    !pip install .[image,example]
    %cd examples/multisite_neuroimg_adapt
else:
    print('Not running on CoLab')

Not running on CoLab


This imports required modules.

In [ ]:
import os
import numpy as np
import pandas as pd
from config import get_cfg_defaults
from nilearn.connectome import ConnectivityMeasure
from nilearn.datasets import fetch_abide_pcp
from sklearn.linear_model import RidgeClassifier

import kale.utils.seed as seed
from kale.evaluate import cross_validation
from kale.pipeline.multi_domain_adapter import CoIRLS

In [ ]:
# Path to `.yaml` config file
cfg_path = "configs/tutorial.yaml" 
cfg = get_cfg_defaults()
cfg.merge_from_file(cfg_path)
cfg.freeze()
seed.set_seed(cfg.SOLVER.SEED)
print(cfg)

## Data Preparation

### Fetch ABIDE fMRI timeseries

In [ ]:
root_dir = cfg.DATASET.ROOT
pipeline = cfg.DATASET.PIPELINE  # fmri pre-processing pipeline
atlas = cfg.DATASET.ATLAS
site_ids = cfg.DATASET.SITE_IDS
abide = fetch_abide_pcp(data_dir=root_dir, pipeline=pipeline,
                        band_pass_filtering=True, global_signal_regression=False,
                        derivatives=atlas, quality_checked=False,
                        SITE_ID=site_ids,
                        verbose=0)

### Read Phenotypic data

In [ ]:
pheno_file = os.path.join(cfg.DATASET.ROOT, "ABIDE_pcp/Phenotypic_V1_0b_preprocessed1.csv")
pheno_info = pd.read_csv(pheno_file, index_col=0)

View Phenotypic data

In [ ]:
pheno_info.head()

### Read timeseries from files

In [ ]:
data_dir = os.path.join(root_dir, "ABIDE_pcp/%s/filt_noglobal" % pipeline)
use_idx = []
time_series = []
for i in pheno_info.index:
    data_file_name = "%s_%s.1D" % (pheno_info.loc[i, "FILE_ID"], atlas)
    data_path = os.path.join(data_dir, data_file_name)
    if os.path.exists(data_path):
        time_series.append(np.loadtxt(data_path, skiprows=0))
        use_idx.append(i)

Use "DX_GROUP" (autism vs control) as labels, and "SITE_ID" as covariates

In [ ]:
pheno = pheno_info.loc[use_idx, ["SITE_ID", "DX_GROUP"]].reset_index(drop=True)

## Extracting Brain Networks Features

In [ ]:
correlation_measure = ConnectivityMeasure(kind='correlation', vectorize=True)
brain_networks = correlation_measure.fit_transform(time_series)

## Machine Learning for Multi-site Data

### Cross validation Pipeline

### Baseline

In [ ]:
estimator = RidgeClassifier()
results = cross_validation.leave_one_group_out(
    brain_networks, pheno["DX_GROUP"].values, pheno["SITE_ID"].values, estimator
)

In [ ]:
pd.DataFrame.from_dict(results)

### Domain Adaptation

In [ ]:
estimator = CoIRLS(kernel=cfg.MODEL.KERNEL, lambda_=cfg.MODEL.LAMBDA_, alpha=cfg.MODEL.ALPHA)
results = cross_validation.leave_one_group_out(
  brain_networks, pheno["DX_GROUP"].values, pheno["SITE_ID"].values, estimator, use_domain_adaptation=True
)

In [ ]:
pd.DataFrame.from_dict(results)